# 19대 대선 결과 분석

### Selenium과 Beautiful Soup을 이용한 데이터 획득 준비 작업

In [1]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

#폰트 일단 생략

# path = "c:/Windows/Fonts/malgun.ttf"
# from matplotlib import font_manager, rc
# if platform.system() == 'Darwin':
#     rc('font', family='AppleGothic')
# elif platform.system() == 'Windows':
#     font_name = font_manager.FontProperties(fname=path).get_name()
#     rc('font', family=font_name)
# else:
#     print('Unknown system... sorry~~~~')    

# plt.rcParams['axes.unicode_minus'] = False

In [2]:
from selenium import webdriver
import time

In [3]:
# 크롬 드라이버 접속
driver = webdriver.Chrome('/Users/HojinChoi/Desktop/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

In [4]:
# 대통령선거라는 글자부분 클릭. 크롬 개발자 도구로 코드를 확인하면 id가 electionType1으로 나타남.
driver.find_element_by_id("electionType1").click()

In [5]:
# 아래 선택항목에서 제19대 선택. 크롬 개발자 도구에서 해당 위치 id는 electionName라고 나타남.
driver.find_element_by_id("electionName").send_keys("제19대")

In [6]:
# 그리고 나타나는 선택에서 대통령선거 선택.
driver.find_element_by_id("electionCode").send_keys("대통령선거")

In [7]:
# 이제 나타나는 시도 항목에서 선택 부분의 XPath를 찾고, 해당 리스트를 가져옴.
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

### 19대 대선 개표 결과 데이터 획득하기¶

- 지역을 선택하면 득표수에 득표율이 괄호()로 함께 나타난다. ex) 더불어민주당 문재인 2,781,345 (42.34)
- (를 기준으로 왼쪽 숫자만 얻어서, 콤마(,)를 제거하고, float형으로 변경하는 함수 **get_num**

In [8]:
import re

def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

- wait.until 함수 : 검색 버튼이 클릭 가능할 때까지 기다리는 기능
-  move_sido 함수 : 광역시도 이름을 리스트에 전송하고 검색 버튼을 누르는 역할

In [9]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH,make_xpath)))
    driver.find_element_by_xpath(make_xpath).click()

- append_data 함수 : 빈 내용으로 미리 준비한 DataFrame에 append 명령으로 읽은 데이터를 하나씩 추가하는 기능
- 이 함수가 실행되면 전체 투표 수, 문재인 후보, 홍준표 후보, 안철수 후보의 득표 수가 추가됨

In [10]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(get_num(each[2]))
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

In [11]:
# 미리 변수 생성
election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],    
                       'ahn' : [] }

In [12]:
# html 태그 중 table로 되어 있는 부분을 Beautiful Soup으로 읽어 padnas의 read_html로 읽음
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
        move_sido(each_sido)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find('table')
    
        df = pd.read_html(str(table))
    
        append_data(df, each_sido, election_result_raw)

TypeError: expected string or bytes-like object